In [4]:
import io

import requests
import docx

In [5]:
def clean_line(line):
    line = line.strip()
    line = line.strip('\uFEFF')
    return line

def read_faq(file_id):
    url = f'https://docs.google.com/document/d/{file_id}/export?format=docx'
    
    response = requests.get(url)
    response.raise_for_status()
    
    with io.BytesIO(response.content) as f_in:
        doc = docx.Document(f_in)

    questions = []

    question_heading_style = 'heading 2'
    section_heading_style = 'heading 1'
    
    heading_id = ''
    section_title = ''
    question_title = ''
    answer_text_so_far = ''
     
    for p in doc.paragraphs:
        style = p.style.name.lower()
        p_text = clean_line(p.text)
    
        if len(p_text) == 0:
            continue
    
        if style == section_heading_style:
            section_title = p_text
            continue
    
        if style == question_heading_style:
            answer_text_so_far = answer_text_so_far.strip()
            if answer_text_so_far != '' and section_title != '' and question_title != '':
                questions.append({
                    'text': answer_text_so_far,
                    'section': section_title,
                    'question': question_title,
                })
                answer_text_so_far = ''
    
            question_title = p_text
            continue
        
        answer_text_so_far += '\n' + p_text
    
    answer_text_so_far = answer_text_so_far.strip()
    if answer_text_so_far != '' and section_title != '' and question_title != '':
        questions.append({
            'text': answer_text_so_far,
            'section': section_title,
            'question': question_title,
        })

    return questions

In [8]:
faq_documents = {
    'llm-zoomcamp': '1qZjwHkvP0lXHiE4zdbWyUXSVfmVGzougDD6N37bat3E',
}

In [9]:
documents = []

for course, file_id in faq_documents.items():
    print(course)
    course_documents = read_faq(file_id)
    documents.append({'course': course, 'documents': course_documents})

llm-zoomcamp


In [10]:
len(documents)

1

In [33]:
from elasticsearch import Elasticsearch

In [34]:
es_client = Elasticsearch('http://localhost:9200') 

In [35]:
es_client

<Elasticsearch(['http://localhost:9200'])>

In [38]:
query="When is the next cohort?"

search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["text", "section", "question^3","course","document_id"],
                    "type": "best_fields"
                }
            }
        }
    }
}

response = es_client.search(index='documents_20240816_0828', body=search_query)

result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])
    


In [39]:
result_docs

[{'text': 'Summer 2026.',
  'section': 'General course-related questions',
  'question': 'When is the next cohort?',
  'course': 'llm-zoomcamp',
  'document_id': 'b6fa77f3'},
 {'text': 'Cosine similarity is a measure used to calculate the similarity between two non-zero vectors, often used in text analysis to determine how similar two documents are based on their content. This metric computes the cosine of the angle between two vectors, which are typically word counts or TF-IDF values of the documents. The cosine similarity value ranges from -1 to 1, where 1 indicates that the vectors are identical, 0 indicates that the vectors are orthogonal (no similarity), and -1 represents completely opposite vectors.',
  'section': 'Module 3: X',
  'question': 'What is the cosine similarity?',
  'course': 'llm-zoomcamp',
  'document_id': 'ee355823'},
 {'text': 'The error indicates that you have not changed all instances of “employee_handbook” to “homework” in your pipeline settings',
  'section': 